In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd # 2.x to support pyarrow
import pyarrow as pa
import openpyxl # for reading xlsx with structure
import numpy as np
import plotly.express as px
import math
import re
from datetime import timedelta, datetime
import os

import tensorflow as tf
from tensorflow import keras

In [ ]:
x_structure = pd.read_parquet('../data/pipeline/x_structure.parquet')
y_structure = pd.read_parquet('../data/pipeline/y_structure.parquet')

In [ ]:
x_train_pretty_1h = pd.read_parquet('../data/pipeline/x_train_pretty_1h.parquet')
x_test_pretty_1h = pd.read_parquet('../data/pipeline/x_test_pretty_1h.parquet')
y_train_tte_1h = pd.read_parquet('../data/pipeline/y_train_tte_1h.parquet')

In [ ]:
left = x_train_pretty_1h.set_index(['ИМЯ МАШИНЫ', 'DT']).astype('float64').sort_index().ffill()[x_structure.index]
left_test = x_test_pretty_1h.set_index(['ИМЯ МАШИНЫ', 'DT']).astype('float64').sort_index().ffill()[x_structure.index]
left_test_raw = x_test_pretty_1h.set_index(['ИМЯ МАШИНЫ', 'DT']).astype('float64').sort_index()[x_structure.index]

left_stats = left.describe()

left = left / left_stats.loc['std']
left_test = left_test / left_stats.loc['std']

MAX_TTE = 31 * 24 * 60 * 60

In [ ]:
# model = keras.Sequential([
#     keras.Input((24*7, len(x_structure.index))),
#     keras.layers.Dense(len(y_structure.index)*3, activation='relu'),
#     keras.layers.Dense(len(y_structure.index)*2, activation='relu'),
#     keras.layers.Dense(len(y_structure.index), activation='sigmoid'),
# ])
# model.summary()

In [ ]:
index = left.index.levels[1]
split_left = index[int(0.7 * len(index))]
split_right = index[int(0.7 * len(index)) + 1]

left_train = left.loc[pd.IndexSlice[:, :split_left], :]
left_val = left.loc[pd.IndexSlice[:, split_right:], :]

display((len(left_train), len(left_val)))

In [ ]:
# @WIP: Гидра

# right = y_train_tte_1h.set_index(['ИМЯ МАШИНЫ', 'DT']).astype('float64') / MAX_TTE
# right_train = right.loc[pd.IndexSlice[:, :split_left], :]
# right_val = right.loc[pd.IndexSlice[:, split_right:], :]

In [ ]:
sequence_length = 24 * 7
sequence_stride = 24

def prepare_batches(data):
    batches = None
    for machine in x_structure.columns:
        seq = data.loc[machine].sort_index().astype('float64').ffill().fillna(0)
        X = seq[x_structure.index]
        Y = seq.drop(x_structure.index, axis=1)

        inputs = keras.utils.timeseries_dataset_from_array(
            X, None, batch_size=64,
            sequence_length=sequence_length,
            sequence_stride=sequence_stride,
            seed=1337,
        )
        targets = keras.utils.timeseries_dataset_from_array(
            Y, None, batch_size=64,
            sequence_length=sequence_length,
            sequence_stride=sequence_stride,
            seed=1337,
        )
        machine_examples = tf.data.Dataset.zip((inputs, targets))

        if batches is None:
            batches = machine_examples
        else:
            batches = batches.concatenate(machine_examples)
    return batches

from sklearn.metrics import mean_squared_error

loss_a = 30
metric_a = 30

def wrmse_loss(y_true, y_pred):
    loss = keras.backend.square(y_pred - y_true)  # (batch_size, 2)
    t = y_true
    a = loss_a
    weights = 1 / (keras.backend.clip(a * t, 1, a)) # @NOTE: clip чтобы не получать бесконечности когда t стремится к 0
    loss *= weights
    loss = keras.backend.mean(loss, axis=1)        # (batch_size,)
    return loss

def wrmse_metric(y_true, y_pred):
    t = y_true
    a = metric_a
    weights = 1 / ((a * t).clip(lower=1)) # @NOTE: clip чтобы не получать бесконечности когда t стремится к 0
    e = y_pred - y_true
    se = e ** 2
    wse = se * weights
    wmse: pd.DataFrame = wse.mean()
    wrmse = wmse ** 0.5
    average_wrmse = wrmse
    # average_wrmse = wrmse.mean() # @WIP: Гидра
    return wrmse[0]

def train_model(target_place_name_and_type):
    model_path = f'../dist/models_v3/{target_place_name_and_type}.h5'

    # if os.path.isfile(model_path):
    #     return

    right = y_train_tte_1h.set_index(['ИМЯ МАШИНЫ', 'DT'])[[target_place_name_and_type]].astype('float64') / MAX_TTE
    right_train = right.loc[pd.IndexSlice[:, :split_left], :]
    right_val = right.loc[pd.IndexSlice[:, split_right:], :]

    train_data = pd.merge(left_train, right_train, left_index=True, right_index=True)
    val_data = pd.merge(left_val, right_val, left_index=True, right_index=True)

    train_batches = prepare_batches(train_data)
    val_batches = prepare_batches(val_data)

    model = keras.Sequential([
        keras.Input((sequence_length, len(x_structure.index))),
        # keras.layers.Dense(sequence_length * len(x_structure.index) / 4, activation='leaky_relu'),
        keras.layers.Dense(24*3, activation='leaky_relu'),
        keras.layers.Dense(1, activation='sigmoid'),
    ])

    # model.compile(loss=wrmse_loss, optimizer='adam', metrics=["mae"])
    model.compile(loss='mse', optimizer='adam', metrics=["mae"])
    # model.summary()
    #
    print(str(datetime.now()) + ' -- ' + target_place_name_and_type)

    train_batch_idx = 6
    train_sample_idx = 60
    x_train_demos, y_train_demos = list(train_batches)[train_batch_idx]
    x_train_demo = x_train_demos[train_sample_idx]
    y_train_demo = y_train_demos[train_sample_idx]

    val_batch_idx = 13
    val_sample_idx = 6
    x_val_demos, y_val_demos = list(val_batches)[val_batch_idx]
    x_val_demo = x_val_demos[val_sample_idx]
    y_val_demo = y_val_demos[val_sample_idx]

    # history = model.fit(train_batches, validation_data=val_batches, epochs=0)
    history = model.fit(train_batches, validation_data=val_batches, epochs=100)
    # history = model.fit(np.array(x_train_demos), np.array(y_train_demos), validation_data=(x_val_demos, y_val_demos), epochs=1000)
    # history = model.fit(np.array([x_train_demo]), np.array([y_train_demo]), epochs=1000)

    # model.save(model_path, save_format='h5')
    # return model, history, (x_train_demo, y_train_demo)
    return model, history, (x_val_demo, y_val_demo)

model_v1 = keras.models.load_model(f'../dist/models/РОТОР TTE M3.h5', compile=False)
model_v3, history, (x_demo, y_demo) = train_model('РОТОР TTE M3')

t_demo_v1 = model_v1.predict(np.array([x_demo]))[0]
t_demo_v3 = model_v3.predict(np.array([x_demo]))[0]
t_one = np.ones(t_demo_v3.shape)
t_zero = np.zeros(t_demo_v3.shape)
t_half = np.zeros(t_demo_v3.shape) + 0.5
t_const = np.zeros(t_demo_v3.shape) + 0.15
t_random = np.random.uniform(size=t_demo_v3.shape)

if 'val_loss' in pd.DataFrame(history.history).columns:
    px.line(history.history, y=['loss', 'val_loss']).show()
    px.line(history.history, y=['mae', 'val_mae']).show()
else:
    px.line(history.history, y=['loss']).show()
    px.line(history.history, y=['mae']).show()

score_demo_v1 = wrmse_metric(pd.DataFrame(y_demo), pd.DataFrame(t_demo_v1))
score_demo_v3 = wrmse_metric(pd.DataFrame(y_demo), pd.DataFrame(t_demo_v3))
score_one = wrmse_metric(pd.DataFrame(y_demo), pd.DataFrame(t_one))
score_half = wrmse_metric(pd.DataFrame(y_demo), pd.DataFrame(t_half))
score_const = wrmse_metric(pd.DataFrame(y_demo), pd.DataFrame(t_const))
score_zero = wrmse_metric(pd.DataFrame(y_demo), pd.DataFrame(t_zero))
score_random = wrmse_metric(pd.DataFrame(y_demo), pd.DataFrame(t_random))
print(
    f'score_demo_v1 = {score_demo_v1}\n'
    f'score_demo_v3 = {score_demo_v3}\n'
    f'score_one = {score_one}\n'
    f'score_half = {score_half}\n'
    f'score_const = {score_const}\n'
    f'score_zero = {score_zero}\n'
    f'score_random = {score_random}\n'
)
px.line(pd.DataFrame(x_demo)).show()

out_x = pd.DataFrame(x_demo, columns=x_structure.index)
out_y = pd.DataFrame(y_demo, columns=['y']) \
    .merge(pd.DataFrame(t_demo_v1, columns=['t_demo_v1']), left_index=True, right_index=True) \
    .merge(pd.DataFrame(t_demo_v3, columns=['t_demo_v3']), left_index=True, right_index=True)

px.line(out_x)
px.line(out_y)

In [ ]:
def validate_model(target_place_name_and_type, machine):
    model = keras.models.load_model(f'../dist/models/{target_place_name_and_type}.h5', compile=False)
    right = y_train_tte_1h.set_index(['ИМЯ МАШИНЫ', 'DT'])[[target_place_name_and_type]].astype('float64') / MAX_TTE

    input_df = left_train.loc[machine].astype('float64').ffill()
    result = pd.DataFrame(index=right.loc[machine].index)
    result['ПРОГНОЗ'] = 0

    x = 0
    while x < len(input_df) - 24*7 - 24:
        window = input_df.iloc[x:x+24*7]
        if len(window) < 24:
            break
        input = np.array([window])
        output = model.predict(input, verbose=0)
        result['ПРОГНОЗ'].iloc[x+24*7:x+24*7+24] = output[0][-24:].reshape((24))

        print("{:3.2f}%".format(100 * x / len(input_df)))
        x += 24

model_v1 = keras.models.load_model(f'../dist/models/УЛИТА TTE M3.h5', compile=False)
validate_model('УЛИТА TTE M3', 'ЭКСГАУСТЕР А/М №4')

In [ ]:
submission1_ref = pd.read_excel('../data/source/sample_submission/submission_1.xlsx', index_col=0)
submission2_ref = pd.read_parquet('../data/source/sample_submission/sample_submission_2.parquet')
submission3_ref = pd.read_parquet('../data/source/sample_submission/sample_submission_3.parquet')

In [ ]:
x_test = pd.read_parquet('../data/source/X_test.parquet')

In [ ]:
# submission1 = submission1_ref.copy()
# submission1['machine'] = np.NaN
# submission1['tm'] = np.NaN
# submission2 = pd.DataFrame(index=x_test.index, columns=submission2_ref.columns)
# submission3 = pd.DataFrame(index=x_test.index, columns=submission3_ref.columns)

def apply_model(target_place_name):
    submission2_slice_path = f'../dist/submission2/{place}.parquet'
    submission3_slice_path = f'../dist/submission3/{place}.parquet'
    if os.path.isfile(submission2_slice_path) and os.path.isfile(submission3_slice_path):
        return

    submission2_slice = pd.DataFrame(index=x_test.index)
    submission3_slice = pd.DataFrame(index=x_test.index)

    print(str(datetime.now()) + ' -- ' + target_place_name)
    for machine in y_structure.columns:
        prediction_field_m1 = f'{target_place_name} TTE M1'
        prediction_field_m3 = f'{target_place_name} TTE M3'
        model_m1 = keras.models.load_model(f'../dist/models/{prediction_field_m1}.h5', compile=False)
        model_m3 = keras.models.load_model(f'../dist/models/{prediction_field_m3}.h5', compile=False)
        y_name = y_structure[machine].loc[target_place_name]
        if y_name not in submission2_ref.columns:
            continue

        input_df = left_test.loc[machine].astype('float64').ffill()
        input_raw_df = left_test_raw.loc[machine].astype('float64')
        result = pd.DataFrame(index=left_test.loc[machine].index, columns=[prediction_field_m1, prediction_field_m3])

        x = 0
        while x < len(input_df) - 24*7 - 24:
            window = input_df.iloc[x:x+24*7]
            if len(window) < 24:
                break
            input = np.array([window])
            output_m1 = model_m1.predict(input, verbose=0)
            output_m3 = model_m3.predict(input, verbose=0)
            result[prediction_field_m1].iloc[x+24*7:x+24*7+24] = output_m1[0][-24:].reshape((24))
            result[prediction_field_m3].iloc[x+24*7:x+24*7+24] = output_m3[0][-24:].reshape((24))
            x += 24

        upsampled = result.rolling(72).mean().resample('10s').interpolate().fillna(1)
        submission2_slice[y_name] = upsampled[prediction_field_m3].map(lambda x: 1 if x < 0.2 else 0)
        submission3_slice[y_name] = upsampled[prediction_field_m1] * MAX_TTE

    submission2_slice.to_parquet(submission2_slice_path)
    submission3_slice.to_parquet(submission3_slice_path)

for place in y_structure.index:
    apply_model(place)

# for place in ['РОТОР']:
#     for machine in ['ЭКСГАУСТЕР А/М №4']:
#         apply_model(place, machine)